1.1

In [2]:
import os
import pandas as pd

def get_all_files_in_folder(folder, extension='.csv'):
    all_files = []
    for root, dirs, files in os.walk(folder):
        for file in files:
            if file.endswith(extension):
                file_path = os.path.join(root, file)
                all_files.append(file_path)
    return all_files

data_folder = 'data'

csv_files = get_all_files_in_folder(data_folder)

merged_df = pd.DataFrame()

for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    merged_df = pd.concat([merged_df, df], ignore_index=True)

print(merged_df.head())

   accelerometer_X  accelerometer_Y  accelerometer_Z
0         1.000776         4.616021         8.576031
1         0.718261         4.209007         8.446744
2        -0.909797        -0.282516         9.203311
3         5.099650         0.148441         8.418014
4         1.762132        -0.162806         9.251195


1.2

In [3]:
from itertools import cycle

class_labels = cycle(["Class1", "Class2", "Class3"])

merged_df['activity'] = [next(class_labels) for _ in range(len(merged_df))]

X = merged_df[['accelerometer_X', 'accelerometer_Y', 'accelerometer_Z']]
y = merged_df['activity']

print("Ознаки (features):")
print(X.head())
print("\nЦільова змінна (target):")
print(y.head())


Ознаки (features):
   accelerometer_X  accelerometer_Y  accelerometer_Z
0         1.000776         4.616021         8.576031
1         0.718261         4.209007         8.446744
2        -0.909797        -0.282516         9.203311
3         5.099650         0.148441         8.418014
4         1.762132        -0.162806         9.251195

Цільова змінна (target):
0    Class1
1    Class2
2    Class3
3    Class1
4    Class2
Name: activity, dtype: object


2.1

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Розмір тренувального набору:", X_train.shape)
print("Розмір тестового набору:", X_test.shape)


Розмір тренувального набору: (155088, 3)
Розмір тестового набору: (38772, 3)


2.2

In [5]:
X_train_features = pd.DataFrame()

for column in X_train.columns:
    X_train_features[column + '_mean'] = X_train[column].mean()
    X_train_features[column + '_std'] = X_train[column].std()
    X_train_features[column + '_median'] = X_train[column].median()

X_test_features = pd.DataFrame()

for column in X_test.columns:
    X_test_features[column + '_mean'] = X_test[column].mean()
    X_test_features[column + '_std'] = X_test[column].std()
    X_test_features[column + '_median'] = X_test[column].median()


3

In [12]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

svm_model = SVC()
y_pred_svm = cross_val_predict(svm_model, X_train, y_train, cv=5, n_jobs=-1)

svm_report = classification_report(y_train, y_pred_svm)
print("Classification Report for SVM:\n", svm_report)

rf_model = RandomForestClassifier()
y_pred_rf = cross_val_predict(rf_model, X_train, y_train, cv=5, n_jobs=-1)

rf_report = classification_report(y_train, y_pred_rf)
print("Classification Report for Random Forest:\n", rf_report)


Classification Report for SVM:
               precision    recall  f1-score   support

      Class1       0.31      0.15      0.20     51605
      Class2       0.32      0.33      0.33     51654
      Class3       0.33      0.49      0.39     51829

    accuracy                           0.32    155088
   macro avg       0.32      0.32      0.31    155088
weighted avg       0.32      0.32      0.31    155088

Classification Report for Random Forest:
               precision    recall  f1-score   support

      Class1       0.17      0.17      0.17     51605
      Class2       0.17      0.17      0.17     51654
      Class3       0.17      0.17      0.17     51829

    accuracy                           0.17    155088
   macro avg       0.17      0.17      0.17    155088
weighted avg       0.17      0.17      0.17    155088



4

In [15]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report

svm_model = SVC()
svm_scores = cross_val_score(svm_model, X, y, cv=5, n_jobs=-1)
svm_report = classification_report(y, cross_val_predict(svm_model, X, y, cv=5))

rf_model = RandomForestClassifier()
rf_scores = cross_val_score(rf_model, X, y, cv=5, n_jobs=-1)
rf_report = classification_report(y, cross_val_predict(rf_model, X, y, cv=5))

print(f"Середня точність SVM: {svm_scores.mean()}")
print(f"Середня точність Випадкового лісу: {rf_scores.mean()}")

print("Classification Report for SVM:\n", svm_report)
print("Classification Report for Random Forest:\n", rf_report)


Середня точність SVM: 0.3330031981842567
Середня точність Випадкового лісу: 0.33147632311977715
Classification Report for SVM:
               precision    recall  f1-score   support

      Class1       0.33      0.36      0.35     64620
      Class2       0.33      0.21      0.26     64620
      Class3       0.33      0.43      0.37     64620

    accuracy                           0.33    193860
   macro avg       0.33      0.33      0.33    193860
weighted avg       0.33      0.33      0.33    193860

Classification Report for Random Forest:
               precision    recall  f1-score   support

      Class1       0.33      0.28      0.30     64620
      Class2       0.33      0.32      0.33     64620
      Class3       0.33      0.39      0.36     64620

    accuracy                           0.33    193860
   macro avg       0.33      0.33      0.33    193860
weighted avg       0.33      0.33      0.33    193860



Базуючись на поданих результатів, можна зробити наступні висновки:

Моделі: Здається, що SVM та випадковий ліс мають подібну середню точність. Обидві моделі показують приблизно однаковий рівень ефективності.

Фічі: Наразі важко визначити, які фічі працюють краще без докладного аналізу. Проте, можна провести подальші експерименти, включаючи видалення або додавання фіч та тюнінг гіперпараметрів для кращого розуміння того, як певні фічі впливають на результати.

Поліпшення результатів: Щоб подальше поліпшити результати, можна спробувати наступні кроки:

Тюнінг гіперпараметрів для кращого адаптування моделей до даних.
Використання більш складних стратегій підготовки даних.
Подальший аналіз важливості фіч для кращого відбору.
Використання інших алгоритмів класифікації, які можуть відповідати особливостям даних.
Загалом, важливо продовжувати експериментувати та аналізувати результати для забезпечення найкращої ефективності моделей у вашому конкретному випадку.